In [1]:
#Berechnen der Mittelwerte über alle Bleche für jeden Zeitstempel, jeden Datenpunkt 
import numpy as np
from scipy.stats import t
import scipy.stats as st
from Read_DataframeAll import Dataframe_All
import pandas as pd

df = Dataframe_All() #Hereinladen aller DataFrames pro Sensor

Mittelwert = {}
Standardabweichung = {}
Konfidenzintervalle = {}

Konfidenzniveau = 0.95

# Berechne die Mittelwerte und Standardabweichung über jeden Sensor und Kraft pro Sensor (also 8 mal insgesamt) und speichere diese in den oben definierten Dictionaries
for Sensor in df:
    # Begrenzung auf 13.000 Einträge, da ab dort die Versuche beendet sind
    df[Sensor] = df[Sensor].head(13000)
    # Berechnung des Mittelwerts jedes einzelnen Dataframes
    Mittelwert_Sensor = df[Sensor].mean(axis=1)
    # Berechnung der Standardabweichung jedes einzelnen Dataframes
    Standardabweichung_Sensor = df[Sensor].std(axis=1)
    #print(Mittelwert_Sensor)
    # Speicherung der Mittelwerte und Standardabweichung im Dictionary
    Mittelwert[Sensor] = Mittelwert_Sensor
    Standardabweichung[Sensor] = Standardabweichung_Sensor
    #print(Standardabweichung[Sensor])
    
    # Berechnung der Konfidenzintervalle 
    n = df[Sensor].shape[1] # Definiere die Länge (Anzahl an Samples) zur Berechnung des Konfidenzintervalls 
    print(n)
    df1 = n-1
    # Bestimme den Z-Critical Value (z-transformierten Intervallgrenzen) durch t.ppf und nutze dann die Formel für die Berechnung des Intervalls. Der z Value kann auch aus Tabellen abgelesen werden    
    # # Man könnte auch st.means verwenden um den Standard Error zu bestimmen anstatt es händisch zu berechnen(Funktioniert in diesem Fall nicht wirklich)
    z = t.ppf((1+ Konfidenzniveau)/ 2, df1)
    print(z)
    Interval = z * (Standardabweichung[Sensor]/np.sqrt(n))
    Untere_Grenze = Mittelwert[Sensor] - Interval
    Obere_Grenze = Mittelwert[Sensor] + Interval
    Konfidenzintervalle[Sensor] = pd.DataFrame({'Untere_Grenze': Untere_Grenze, 'Obere_Grenze':Obere_Grenze})
      
#print(Konfidenzintervalle['1-OW-OS Lateral Force'])
   
#print(t.ppf((1+Konfidenzniveau)/2, df))
#print(st.sem(SensorDataFrames['2-OW-RS Axial Force']))
#print((Standardabweichung['2-OW-RS Axial Force']/ np.sqrt(n)))
#print(Mittelwert['2-OW-RS Axial Force'])

#Mittelwert_1_OW_OS_Axial.plot()

#plt.show()

Anzahl der Spalten für Sensor 1-OW-OS Lateral Force: 152
Anzahl der Spalten für Sensor 1-OW-OS Axial Force: 152
Anzahl der Spalten für Sensor 2-OW-RS Lateral Force: 152
Anzahl der Spalten für Sensor 2-OW-RS Axial Force: 152
Anzahl der Spalten für Sensor 3-UW-RS Lateral Force: 152
Anzahl der Spalten für Sensor 3-UW-RS Axial Force: 152
Anzahl der Spalten für Sensor 4-UW-OS Lateral Force: 152
Anzahl der Spalten für Sensor 4-UW-OS Axial Force: 152
152
1.9757989238085503
152
1.9757989238085503
152
1.9757989238085503
152
1.9757989238085503
152
1.9757989238085503
152
1.9757989238085503
152
1.9757989238085503
152
1.9757989238085503


In [7]:
#Plots

import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import os 

# Erstellung der Graphen für Mittelwert und Konfidenzintervall aller Sensoren und Kräfte

Sicherungsordner = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\15_Plots\Plots_Konfidenzintervall95%'
os.makedirs(Sicherungsordner, exist_ok=True)

for Sensor, (Untere_Grenze, Obere_Grenze) in Konfidenzintervalle.items():
    
    Konfidenzintervalle[Sensor].reset_index(drop=True, inplace = True) # Index reset da die Funktion fill_between Probleme gemacht hat (Index falscher typ)
    #Konfidenzintervalle[Sensor] = Konfidenzintervalle[Sensor].iloc[1500:] #Abschneiden des Einlaufes
    #Mittelwert[Sensor] =Mittelwert[Sensor].iloc[1500:]
    
    plt.figure(figsize=(10,6))
    
    Untere_Grenze1 = Konfidenzintervalle[Sensor]['Untere_Grenze'] 
    Obere_Grenze1 = Konfidenzintervalle[Sensor]['Obere_Grenze']

    plt.fill_between(Konfidenzintervalle[Sensor].index, Untere_Grenze1, Obere_Grenze1, color='lightblue', alpha=0.9, label= 'Konfidenzintervall' )
    plt.plot(Konfidenzintervalle[Sensor].index, Mittelwert[Sensor], label = 'Mittelwert' , color='darkblue', linewidth= 0.2, linestyle='--')

    plt.xlabel('Datenpunkte', fontsize=16, labelpad=12)
    plt.ylabel('Kraft in Newton', fontsize=16, labelpad=12)
    plt.title(f'Mittelwert und 95% Konfidenzintervall für Sensor {Sensor}',fontsize=17, pad=12)
    plt.legend(loc ='upper right', framealpha=1, edgecolor='black', fontsize=20)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.tight_layout()
    
    # Seperates Festlegen der Linienbreite in der Legende, da der Mittelwert ansonsten nicht in der übergebenen Farbe dargstellt wird 
    leg = plt.legend()
    for line in leg.get_lines():
        line.set_linewidth(2)  # Festlegen der Linienbreite auf 2 für die Legende separat zum Graph

    plt.savefig(os.path.join(Sicherungsordner, f'{Sensor}_Plot.svg'), format='svg')
    plt.close()